In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json
import subprocess

In [2]:
# https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies

In [2]:
tmdb_df = pd.read_csv("D://tmdb_database/TMDB_movie_dataset_v11.csv")
# tmdb_df.head()

In [ ]:
# Convert "release_date" to datetime

tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'])
tmdb_df.dtypes


id                               int64
title                           object
vote_average                   float64
vote_count                       int64
status                          object
release_date            datetime64[ns]
revenue                          int64
runtime                          int64
adult                             bool
backdrop_path                   object
budget                           int64
homepage                        object
imdb_id                         object
original_language               object
original_title                  object
overview                        object
popularity                     float64
poster_path                     object
tagline                         object
genres                          object
production_companies            object
production_countries            object
spoken_languages                object
keywords                        object
dtype: object

In [3]:
df_genres = tmdb_df[['id', 'genres']]
df_genres.head()

,id,genres
0,27205,"Action, Science Fiction, Adventure"
1,157336,"Adventure, Drama, Science Fiction"
2,155,"Drama, Action, Crime, Thriller"
3,19995,"Action, Adventure, Fantasy, Science Fiction"
4,24428,"Science Fiction, Action, Adventure"


In [6]:
df_genres_split = df_genres.assign(genres=tmdb_df['genres'].str.split(','))
# df_genres_split = df_genres.explode('genres')
df_genres_split.head(50)

,id,genres
0,27205,"[Action, Science Fiction, Adventure]"
1,157336,"[Adventure, Drama, Science Fiction]"
2,155,"[Drama, Action, Crime, Thriller]"
3,19995,"[Action, Adventure, Fantasy, Science Fiction]"
4,24428,"[Science Fiction, Action, Adventure]"
5,293660,"[Action, Adventure, Comedy]"
6,299536,"[Adventure, Action, Science Fiction]"
7,550,[Drama]
8,118340,"[Action, Science Fiction, Adventure]"
9,680,"[Thriller, Crime]"


In [ ]:
df_genres_exploded = df_genres_split.explode('genres')
df_genres_exploded.head()



,id,genres
0,27205,Action
0,27205,Science Fiction
0,27205,Adventure
1,157336,Adventure
1,157336,Drama


In [17]:
df_genres_exploded['genres'] = df_genres_exploded['genres'].str.strip()

df_genres_cleaned = df_genres_exploded.dropna()

print(df_genres_cleaned['genres'].unique())

['Action' 'Science Fiction' 'Adventure' 'Drama' 'Crime' 'Thriller'
 'Fantasy' 'Comedy' 'Romance' 'Western' 'Mystery' 'War' 'Animation'
 'Family' 'Horror' 'Music' 'History' 'TV Movie' 'Documentary']


In [ ]:
genre_ref_test = [{1:'Action',
              2:'Science Fiction',
              3:'Adventure',
              4:'Drama',
              5:'Crime',
              6:'Thriller',
              7:'Fantasy',
              8:'Comedy',
              9:'Romance',
              10:'Western',
              11:'Mystery',
              12:'War',
              13:'Animation',
              14:'Family',
              15:'Horror',
              16:'Music',
              17:'History',
              18:'TV Movie',
              19:'Documentary'  
}]

In [ ]:
genre_ref = [{'Action':1,
              'Science Fiction':2}]
             ''' 3:'Adventure',
              4:'Drama',
              5:'Crime',
              6:'Thriller',
              7:'Fantasy',
              8:'Comedy',
              9:'Romance',
              10:'Western',
              11:'Mystery',
              12:'War',
              13:'Animation',
              14:'Family',
              15:'Horror',
              16:'Music',
              17:'History',
              18:'TV Movie',
              19:'Documentary'  '''


In [25]:
df_genres_cleaned['genres'] = df_genres_cleaned['genres'].replace(genre_ref)
df_genres_cleaned.head()

C:\Users\jeffb\AppData\Local\Temp\ipykernel_24776\3532397337.py:1: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df_genres_cleaned['genres'] = df_genres_cleaned['genres'].replace(genre_ref)
C:\Users\jeffb\AppData\Local\Temp\ipykernel_24776\3532397337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genres_cleaned['genres'] = df_genres_cleaned['genres'].replace(genre_ref)


,id,genres
0,27205,Action
0,27205,Science Fiction
0,27205,Adventure
1,157336,Adventure
1,157336,Drama
